### BANK TRANSACTION DATA PIPELINE & FRAUD DETECTION WITH MongoDB 

1. Design the MongoDB Schema
    a. Define the collection and their respective fields.
        - Accounts and credit limits as well as credit balance etc.
        - Banks & Countries and minimum balance for customers and the charges for certain transfers.
        - The exchange rates.
        - Merchants
        - Transactions
2. Populate the accounts, banks and merchants collection according to the validators. Implement minimum balances with different curre
3. Simulate transaction data with logic below.
    a.  For each transaction, check if the sender and receiver are valid accounts.
    b.  If valid, update the sender and receiver balance accordingly and insert into collection.
    c.  Specify the transaction type be it a deposit, atm withdrawal, purchase(online or in-store), cheques(withdrawal or deposit), charge, POS, payment or transfer.
    d. The account should not be below the balance for balance account and not go over for credit accounts.
    e. If not valid, log the error and skip the transaction.
    f. Every transaction should be unique.
    g. Implement the logic to calculate the exchange rate for each transaction based on the exchange rates collection.
    h. Implement the logic to calculate the fees for each transaction based on the banks and countries collection.
4. Data Transformation
5. Data Loading into MongoDB
6. Fraud detection logic/cases.
7. Data Analytics
8. Automation.


In [1]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")

# Define the database
db = client["banking-system"]


In [23]:
import random
primary_accounts = db.accounts.find({"account_category": "Customer"})
primary_account = random.choice(list(primary_accounts))['current_balance']
print(primary_account)

10040.08


In [39]:
POS_PAYMENTS = {
    "restaurant": [
        "dine-in",       # Dining at the restaurant
        "takeaway",      # Taking food away from the restaurant
        "delivery"       # Food delivery service
    ],
    "retail": [
        "in-store",      # Shopping in a physical store
        "online",        # Shopping online
        "curbside"       # Curbside pickup
    ],
    "service": [
        "appointment",   # Scheduled service appointments
        "walk-in",       # Walk-in services
        "subscription"   # Subscription-based services
    ],
    "grocery": [
        "in-store",      # Shopping in a physical grocery store
        "online",        # Online grocery shopping
        "delivery"       # Grocery delivery service
    ],
    "other": [
        "miscellaneous"  # Miscellaneous POS payments
    ]
}

category = random.choice(list(POS_PAYMENTS.keys()))
random_pos_item = random.choice(POS_PAYMENTS[category])
print(f"{category}: {random_pos_item}")

retail: curbside


In [25]:
from datetime import datetime


exchange_rates_doc = {
    "currency_pair": "USD/EUR",
    "exchange_rate": 0.85,
    "timestamp": datetime(2024, 12, 6, 12, 0, 0)  # Convert to datetime
}


db.exchange.insert_one(exchange_rates_doc)




InsertOneResult(ObjectId('6754c1d7f239f2444d6c052c'), acknowledged=True)